In [98]:
#IMPORTS
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import dataUtils as du

In [99]:
# Create the model
class FeedForwardNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = torch.load("Models/Final_models/2hid_32_neu_500epoch_p001lr_norm-11.pth")

# Load gridsearch data for normalisation
(OUR MODEL TAKES TRANSLATIONS THEN ROTATIONS, GRID SEARCH DATA IS ROTATIONS THEN TRANSLATIONS)

In [100]:
path_base = "Tracking/Gridsearch/"
a = 100 #acceleration in steps
hz = 30
samples = 3 #steps in gridsearch
stepsize = 20
full_path = ""
num_meta_rows = 6

#load all data
df = du.load_dataset(a,samples,stepsize,path_base,full_path)
#print(df.head)

# Change rotations to accurate coordinate system and make translations relative to frame 0
df = du.fixCoordinates(df)

#get valid frames (frames without wobeling/ocscilation)
start_offset = 65 #(frame before action)
wanted_frames = samples ** 6 #amount of frames to extract
frame_offset = 2*hz #time between moves
spacing = 10 #how many previous frames that have to be still
t_eps = 0.5 #translational error
r_eps =0.5 #rotational error

df_valid, valid_idx = du.getValidFrames(df,wanted_frames,frame_offset,start_offset,spacing,t_eps,r_eps)
X_norm = df_valid

TOTAL FRAMES: 729 (729)
INVALID FRAMES: 0, VALID FRAMES: 729


In [101]:
#Create list sampling of isolated movements to feed as input to the neural network
def getMoveRange(inc,max_val,move_type):
  cur = -max_val
  moves = []
  while cur <= max_val:
    tmp = [0,0,0,0,0,0]
    tmp[move_type] = cur
    moves.append(tmp)
    cur += inc
  moves.append([0,0,0,0,0,0])
  return moves

x_trans = getMoveRange(0.5,10,0)
y_trans = getMoveRange(0.5,10,1)
z_trans= getMoveRange(0.5,10,2)
x_rot = getMoveRange(0.5,10,3)
y_rot = getMoveRange(0.5,10,4)
z_rot = getMoveRange(0.5,10,5)

In [102]:
#Regularize the input data
def raw_norm(data):
  col = ["X_trans", "Y_trans", "Z_trans", "X_rot", "Y_rot", "Z_rot"]
  data_norm = np.zeros(data.shape)
  for i in range(6):
    data_norm[:,i] = 2 * (data[:,i] - X_norm[col[i]].min()) / (X_norm[col[i]].max() - X_norm[col[i]].min()) - 1
  return data_norm

In [103]:
# Provide input data (head poses, either ABCD data or custom data), given as translation then rotation
data = np.array(x_trans + y_trans + z_trans + x_rot + y_rot + z_rot)
#data = np.load("Data/ABCD/fake_abcd.npy")[0]
data_norm = raw_norm(data)

print(np.min(data_norm),np.max(data_norm))

-1.8072095561730879 1.8050820970039614


In [104]:
data_tensor = torch.tensor(data_norm, dtype=torch.float32) #data_norm

# Combine the input data and target values into a TensorDataset
data_set = TensorDataset(data_tensor)

# Create a DataLoader for batch processing
data_loader = DataLoader(data_set, batch_size=batch_size, shuffle=False)

In [105]:
def toNumpy(tensors):
  arr = [tensor.numpy() for tensor in tensors]
  arr = np.array(arr)
  return arr

In [106]:
# Make setup that takes 1 person and gives cable lengths corresponding to that person

preds = []
# Validation loop
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    for inputs in data_loader:
        outputs = model(inputs[0])
        preds.append(outputs)
        
preds = toNumpy(preds)
print(preds)

[[[ 18.811947    22.28018    -19.02961      7.3995705  -24.937191
   -21.621122  ]]

 [[ 18.428333    22.018263   -18.03156      7.2339354  -24.328722
   -21.175001  ]]

 [[ 17.951824    21.662191   -16.993744     6.973524   -23.70801
   -20.774515  ]]

 ...

 [[-17.737474    16.466751   -23.214205    26.75773    -22.333054
    11.8161955 ]]

 [[-17.447062    16.77441    -23.609985    27.640114   -23.391092
    12.663159  ]]

 [[  1.7363617    1.4619877   -1.898098     0.5658432    0.25961065
    -1.5611482 ]]]


In [107]:
# function which checks if any pose requires cables to be outside a specified range
def inspect_tp_vals(tp,min_val,max_val):
  print(f"MAX {np.max(np.array(tp),axis=0)} MIN {np.min(np.array(tp),axis=0)}")
  for s,i in enumerate(tp):
    for j in i:
      if j > max_val or j < min_val:
        print(i,s)
        break

In [108]:
#round each step and cast them to ints, also include a reset step at last
data_steps = np.round(preds).astype(int)
data_steps = data_steps.reshape((252,6))
tps = list(map(list,data_steps))
inspect_tp_vals(tps,-20,20)

MAX [29 22 27 30 22 28] MIN [ -29  -27  -30  -27  -30 -144]
[19, 22, -19, 7, -25, -22] 0
[18, 22, -18, 7, -24, -21] 1
[18, 22, -17, 7, -24, -21] 2
[17, 21, -16, 7, -23, -20] 3
[17, 21, -15, 6, -22, -20] 4
[16, 20, -14, 6, -22, -20] 5
[16, 19, -13, 5, -21, -19] 6
[-18, -20, 10, -7, 21, 17] 36
[-19, -22, 11, -7, 21, 17] 37
[-19, -23, 12, -8, 21, 17] 38
[-20, -24, 12, -8, 22, 17] 39
[-21, -25, 12, -9, 22, 17] 40
[-29, -21, -30, -26, -29, -31] 42
[-29, -21, -28, -25, -28, -30] 43
[-29, -20, -27, -25, -27, -29] 44
[-29, -20, -26, -24, -26, -29] 45
[-28, -20, -26, -24, -25, -27] 46
[-26, -20, -25, -24, -24, -26] 47
[-24, -19, -24, -23, -23, -24] 48
[-22, -18, -23, -23, -21, -23] 49
[-21, -18, -22, -22, -20, -22] 50
[-19, -17, -20, -21, -18, -21] 51
[20, 18, 18, 21, 16, 18] 72
[21, 19, 19, 22, 17, 19] 73
[22, 19, 20, 23, 17, 20] 74
[23, 19, 21, 24, 18, 20] 75
[24, 19, 22, 25, 19, 21] 76
[25, 19, 23, 25, 19, 22] 77
[26, 20, 24, 26, 20, 23] 78
[27, 20, 25, 27, 20, 24] 79
[28, 20, 26, 28, 20, 26

In [109]:
# Save array to feed to physical system using our protocol
#np.save("Tracking/TPs/name.npy", data_steps)